# Introduction to Conversight Sub Flows

Flows within fows are called as conversight subflows. Tasks and flows can be called within the flow construction. Although the sub flow will have its own set of activities like the construction of tasks, it will also be contributing to the master flow.

The sub-flow can be executed independently and concurrently with other components of the main flow. This distributed execution enhances efficiency by leveraging parallel processing capabilities, enabling multiple tasks to be performed simultaneously. By breaking down the larger flow and distributing its execution through sub-flows, Conversight optimizes the utilization of computing resources.

For ease of understanding, let's design a basic subflow using arithmetic operations. We need to create a task that can be compatible with the sub flow response

In [8]:
from conversight import task, Context

In [9]:
@task(name="square_root", tags=["Arithmetic", "square_root"])
def square_root(x: int, ctx: Context) -> float:
    """Simple square_root task"""
    try:
        import math
        from conversight import SubFlowResponse
        if isinstance(x, SubFlowResponse):
            x = ctx.get(x.response)[0]
        return math.sqrt(x)
    except Exception as e:
        return e

In [10]:
square_root.run(5)

2.23606797749979

In [11]:
description = "Simple square_root task"
square_root.register(libraryName="Arithmetic", description=description, sourceControl="edit", apiAccess=True)

[2023-12-22 17:05:09,288] [INFO] square_root has been successfully registered. The most recent version available is 0.1 !!


The `square_root` task mentioned above was examined to ensure that the argument x type was `SubFlowResponse`. This ensures that the task is appropriate for both int data types and sub flow responses. A `SubFlowResponse` class with a variable response containing the actual reference is always returned when a sub flow is executed.

Now let's convert this task into a flow

In [14]:
from conversight import TaskLibrary, FlowLibrary, Flow, Parameter

In [13]:
tsk = TaskLibrary()

Tasks loaded  !!


In [15]:
flw = FlowLibrary()

Flows loaded !!


In [16]:
with Flow(name="Square_root_Flow", description="A simple square root flow", tags = ["Arithmetic", "square root"]) as flow:
    x = Parameter('x',0)
    y = Parameter('y',0)
    z = Parameter('z',5)

    subflow_response = flw.Arithmetic.Arithmetic_Calculations(x=x, y=y, z=z)
    sqt_response = tsk.Arithmetic.square_root(subflow_response)

[2023-12-22 17:18:07,290] [WARNING] Sub Flow detected, the response will be in the following format 
=> Successful response is SubFlowResponse class with context reference of sub-flow, use ctx.get(SubFlowResponse.response) to retrieve the context response in parent function 
=> Failed response is {status: Failed, message: <failed reason>}


As you can see the in the above flow we used the `Arithmetic_Calculations` as sub flow. This sub flow will run first and returned the response as `SubFlowResponse` to the main flow and it is then used by the `square_root` task in the main flow.

Athena can detect the sub flows and show the warning of how to handle the sub flows within a flow

In [19]:
flow.run(x=10, y=10, z=5)

[2023-12-22 17:23:00,996] [INFO] [Main-Flow]  Received request from AI Workbench, considering as test run..
[2023-12-22 17:23:01,000] [INFO] Context Actor [025aec7cb6da44c7b5759ec9da7c5412] deployed..
calls ==> call
calls ==> call
calls ==> call
calls ==> call
calls ==> call


(runWrapperNotebook pid=850) E1222 17:23:02.827080980     917 chttp2_transport.cc:1160]             ipv4:100.109.95.67:58407: Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings". Current keepalive time (before throttling): 60000ms
(runWrapperNotebook pid=879) E1222 17:23:03.777282244     947 chttp2_transport.cc:1160]             ipv4:100.109.95.67:58407: Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings". Current keepalive time (before throttling): 60000ms


(runWrapperNotebook pid=1196) [2023-12-22 17:23:08,898] [INFO] Sub Flow running on Notebook..
(runWrapperNotebook pid=1196) [2023-12-22 17:23:09,070] [INFO] [Sub-Flow] Received request from AI Workbench, considering as test run..
(runWrapperNotebook pid=1196) [2023-12-22 17:23:09,162] [INFO] Context Actor [d339f9f7dd744b93adc686c37596d76a] deployed..
(runWrapperNotebook pid=1196) calls ==> call
(runWrapperNotebook pid=1196) calls ==> call
(runWrapperNotebook pid=1196) calls ==> call
(runWrapperNotebook pid=1196) calls ==> call
(runWrapperNotebook pid=1196) calls ==> call
[2023-12-22 17:23:17,794] [INFO] [Main-Flow]  Flow response: [4.47213595499958]


[4.47213595499958]

(runWrapperNotebook pid=1196) [2023-12-22 17:23:17,745] [INFO] [Sub-Flow] Flow response: [20.0]
(runWrapperNotebook pid=1196) [2023-12-22 17:23:17,747] [INFO] Setting up sub-flow response (Arithmetic_Calculations-5a02900d6f534338859d07a223b372fa) into context, use ctx.get(SubFlowResponse.response) to retrieve the response
(runWrapperNotebook pid=1196) [2023-12-22 17:23:17,790] [INFO] Context response from Sub-Flow: {response: Arithmetic_Calculations-5a02900d6f534338859d07a223b372fa}


Once we are good with the output, we can register the current flow using the "register" method.

In [20]:
flow.register(libraryName="Arithmetic", flowName="Square_root_with_Sub_Flow", description="A simple square_root flow")

'The flow Square_root_with_Sub_Flow has been registered successfully. Latest version available now is 0.1 !!'

you can use the `reload` function in the `flw` object to get the registered flow

In [22]:
flw.reload()

Flows loaded !!


In [23]:
flw.Arithmetic.Square_root_with_Sub_Flow.run(x=10, y=10, z=5)

[2023-12-22 17:30:52,387] [INFO] [Main-Flow]  Received request from AI Workbench, considering as test run..
[2023-12-22 17:30:52,390] [INFO] Context Actor [8e0e9b191fcd4840a16c0f6f31df1ac5] deployed..
calls ==> call
calls ==> call
calls ==> call
calls ==> call
calls ==> call
(runWrapperNotebook pid=1625) [2023-12-22 17:31:00,380] [INFO] Sub Flow running on Notebook..
(runWrapperNotebook pid=1625) [2023-12-22 17:31:00,633] [INFO] [Sub-Flow] Received request from AI Workbench, considering as test run..
(runWrapperNotebook pid=1625) [2023-12-22 17:31:00,724] [INFO] Context Actor [368b860219814be381c5c82250ac8ffb] deployed..
(runWrapperNotebook pid=1625) calls ==> call
(runWrapperNotebook pid=1625) calls ==> call
(runWrapperNotebook pid=1625) calls ==> call
(runWrapperNotebook pid=1625) calls ==> call
(runWrapperNotebook pid=1625) calls ==> call
(runWrapperNotebook pid=1625) [2023-12-22 17:31:09,317] [INFO] [Sub-Flow] Flow response: [20.0]
(runWrapperNotebook pid=1625) [2023-12-22 17:31:09

[4.47213595499958]

(runWrapperNotebook pid=1625) [2023-12-22 17:31:09,383] [INFO] Context response from Sub-Flow: {response: Arithmetic_Calculations-3251eae44ff3497d9af84109737ebd65}
